In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
from IPython.core.display import display,HTML
try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
    
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout


########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:
import plotly
plotly.__version__

In [2]:
path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
df = pd.read_excel(open(path+'NICO_0511_anonymized.xlsx','rb'),na_values=["NAN","-1"],names = ['folder_id', 'user_id', 'email_domain', 'num_adds', 'num_edits', 'num_deletes', 'num_folder_members', 'major_content_type' ,'major_content_ext',  'age_folder_days' ])



In [ ]:
print "number unique folder_ids:",len(df.folder_id.unique())
print "number unique user_ids:",len(df.user_id.unique())
print "number unique email domains:",len(df.email_domain.unique())
print "number unique major content ext:",len(df.major_content_ext.unique())


In [4]:
for item in df.email_domain.unique():
    print item,"   "

*montana.edu subdomain    
*tamu.edu subdomain    
*uchc.edu subdomain    
*ucla.edu subdomain    
*ucsd.edu subdomain    
*ucsf.edu subdomain    
*uio.no subdomain    
*umn.edu subdomain    
*unm.edu subdomain    
*usc.edu subdomain    
*waseda.ac.jp subdomain    
2014.hull.ac.uk    
2017.icam.fr    
aa.ufl.edu    
aacc.edu    
aalto.fi    
aamu.edu    
aaschool.ac.uk    
aau.at    
aauj.edu    
abbey.bac.edu    
abdn.ac.uk    
aberdeen.ac.uk    
abet.org    
ac.upc.edu    
acad.umass.edu    
academicimpressions.com    
academyart.edu    
acadiau.ca    
acc.org    
accountancy.smu.edu.sg    
ace.tamut.edu    
aces.edu    
acg.edu    
acomedu.org    
acsw.ru.nl    
actx.edu    
acu.edu    
acu.edu.au    
ad.nmsu.edu    
ad.ucsd.edu    
ad.ufl.edu    
ad.umanitoba.ca    
ad.unc.edu    
adams.edu    
adelaide.edu.au    
adelphi.edu    
adjuncts.belmont.edu    
adler.edu    
adm.monash.edu.au    
admin.cam.ac.uk    
admin.fsu.edu    
admin.rochester.edu    
admin.umass.edu    
admin.usf.e

In [34]:
flag_GC="m"  # yes: only plot the GC;    no: i plot the entire network,   m: i select multiple big connected components for plotting




#focus_university="harvard.edu"           #### harvard   missouri      northwestern   berkeley  ucla.  cambridge    ox.ac.uk
list_focus_univ=["missouri.edu","harvard.edu","northwestern.edu", "berkeley","ucla.","cambridge","ox.ac.uk","ucdavis","princeton","uchicago","umass","columbia","unizar.es","duke.edu"]
#list_focus_univ=["northwestern.edu"]


G=nx.Graph()

dict_folder_list_domains={}
dict_folder_list_users={}
dict_user_id_domain={}
dict_user_id_edit={}
dict_user_id_del={}
dict_user_id_add={}



print df.shape
cont=0
for row in df.iterrows():
        #print row[1]
    folder=row[1]["folder_id"]
    user_id=row[1]["user_id"]
        #print folder_id, user_id
    
    email=row[1]["email_domain"]
    
    edits=row[1]["num_edits"]
    adds=row[1]["num_adds"]
    deletes=row[1]["num_deletes"]
   
   
        
    flag_useful_line=0
    for focus_university in list_focus_univ:                 
        if focus_university in email:
            flag_useful_line =1
            break
                 
    if flag_useful_line ==1:
        try: 
            dict_folder_list_users[folder].append(user_id)
        except:
            dict_folder_list_users[folder]=[]
            dict_folder_list_users[folder].append(user_id)
            
        try:    
            dict_folder_list_domains[folder].append(email)
        except:            
            dict_folder_list_domains[folder]=[]
            dict_folder_list_domains[folder].append(email)
            
            
            
            
            
        dict_user_id_domain[user_id]=email.replace(".edu","")                                  
        dict_user_id_edit[user_id]=edits
        dict_user_id_del[user_id]=deletes
        dict_user_id_add[user_id]=adds 
            
             
            
        cont +=1
        if cont % 10000 ==0:
            print cont
            
print "done with the dict., size:",len(dict_folder_list_users)





for folder in dict_folder_list_users:
    dict_folder_list_users[folder]=list(set(dict_folder_list_users[folder]))  # remove possible duplicates

    dict_folder_list_domains[folder]=list(set(dict_folder_list_domains[folder]))
    
    
    lista=dict_folder_list_users[folder]
  
    if len(lista)>1:
        
        lista_pares=itertools.combinations(lista, 2)        
        
        for item in lista_pares:
            e1=item[0]
            e2=item[1]
            G.add_edge(e1,e2)
           # print "added edge:",e1, e2
#     else:   # FOR NOW, I DO NOT ADD THE NODES WITH COLLABORATORS OUTSIDE THE UNIVERSITY DOMAINS
#         user_id=lista[0]
        
#         G.add_node(user_id)






GC_string=""
if flag_GC =="yes":   # i will plot only the GC
    print "selecting only the GC for plotting"
    G = max(nx.connected_component_subgraphs(G), key=len)
    print "  N:", len(G.nodes()),"  L:", len(G.edges())    
    GC_string="_GC"

elif flag_GC =="m":  # i will plot multiple large connected components
    
    print "selecting GC and some other large connected components for plotting"
   
    new_G = max(nx.connected_component_subgraphs(G), key=len)
    print "  N:",len(new_G.nodes()),"  L:",len(new_G.edges())
    lista_comp= reversed(sorted(nx.connected_component_subgraphs(G), key=len))
    
    
    cont_comp=0
    for comp in lista_comp:
        cont_comp +=1
        new_G.add_nodes_from(comp.nodes())
        new_G.add_edges_from(comp.edges())
        print cont_comp,"  N:",len(new_G.nodes()),"  L:",len(new_G.edges())
        if cont_comp >=3:
            break
    
    G= new_G.copy()
    GC_string="_multiple_comp"
            
    
elif flag_GC =="no":  # i will plot the entire structure
    print "selecting entire network for plotting"
    print focus_university, "done.  N:", len(G.nodes()),"  L:", len(G.edges())
    GC = max(nx.connected_component_subgraphs(G), key=len)
    print "\n  GC:     N:", len(GC.nodes()), "  L:", len(GC.edges())



    
    
    
    
    
 

#### the dict is larger than G.nodes(), and the set_node_attribute does not work in that case, so i get a selection of the dict only with the common nodes:
print len(G.nodes()), len(dict_user_id_domain)
new_dict_user_id = {k: dict_user_id_domain[k] for k in set(G.nodes()) & set(dict_user_id_domain.keys())}
print len(G.nodes()), len(new_dict_user_id)
     
# i add the email as node attributes    
nx.set_node_attributes(G, 'email', new_dict_user_id)
############



# print len(G.nodes()), len(dict_user_id_edit)
new_dict_user_id = {k: dict_user_id_edit[k] for k in set(G.nodes()) & set(dict_user_id_edit.keys())}
# print len(G.nodes()), len(new_dict_user_id)
    
# i add the email as node attributes    
nx.set_node_attributes(G, 'edits', new_dict_user_id)
############



# print len(G.nodes()), len(dict_user_id_del)
new_dict_user_id = {k: dict_user_id_del[k] for k in set(G.nodes()) & set(dict_user_id_del.keys())}
# print len(G.nodes()), len(new_dict_user_id)
 
# i add the email as node attributes    
nx.set_node_attributes(G, 'deletes', new_dict_user_id)
############




#print len(G.nodes()), len(dict_user_id_add)
new_dict_user_id = {k: dict_user_id_add[k] for k in set(G.nodes()) & set(dict_user_id_add.keys())}
#print len(G.nodes()), len(new_dict_user_id)

# i add the email as node attributes    
nx.set_node_attributes(G, 'adds', new_dict_user_id)
############
 
       

print list_focus_univ, "done.  N:", len(G.nodes()),"  L:", len(G.edges())  
   
    
    
    
    
    
    
    

########### i relabel the nodes so the plotly works later (otherwise, it doesnt work!)
mapping={}
cont=0
for n in G.nodes():
    mapping[n]=cont
    cont +=1
    
G=nx.relabel_nodes(G,mapping, copy=False)
print "nodes relabeled"
##############################



new_new_dict={}
for node in G.nodes():
    new_new_dict[node]=G.node[node]['email']






(497553, 10)
10000
20000
30000
40000
done with the dict., size: 22538
selecting GC and some other large connected components for plotting
  N: 2442   L: 10017
1   N: 2442   L: 10017
2   N: 3523   L: 15156
3   N: 3773   L: 16476
3773 25638
3773 3773
['missouri.edu', 'harvard.edu', 'northwestern.edu', 'berkeley', 'ucla.', 'cambridge', 'ox.ac.uk', 'ucdavis', 'princeton', 'uchicago', 'umass', 'columbia', 'unizar.es', 'duke.edu'] done.  N: 3773   L: 16476
nodes relabeled


In [36]:


##### i get a layout plotting of the network
dict_pos={}



#dict_pos=nx.random_layout(G,dim=2)     
#dict_pos=nx.fruchterman_reingold_layout(G,dim=2)
#dict_pos=nx.spectral_layout(G,dim=2)
#dict_pos=nx.spring_layout(G, dim=2)


    ##### layout options for   graphviz_layout:
    # dot - "hierarchical" or layered drawings of directed graphs. This is the default tool to use if edges have directionality.
    # neato - "spring model'' layouts.  This is the default tool to use if the graph is not too large (about 100 nodes) and you don't know anything else about it. Neato attempts to minimize a global energy function, which is equivalent to statistical multi-dimensional scaling.
    # fdp - "spring model'' layouts similar to those of neato, but does this by reducing forces rather than working with energy.
    # sfdp - multiscale version of fdp for the layout of large graphs.
    # twopi - radial layouts, after Graham Wills 97. Nodes are placed on concentric circles depending their distance from a given root node.
    # circo - circular layout, after Six and Tollis 99, Kauffman and Wiese 02. This is suitable for certain diagrams of multiple cyclic structures, such as certain telecommunications networks.

dict_pos=graphviz_layout(G, prog='sfdp')

#### i save the resulting  layout as attributes of the nodes themselves
nx.set_node_attributes(G, 'pos', dict_pos)

print "done getting the postion dict,",len(dict_pos)


done getting the postion dict, 3773


In [37]:
pos=dict_pos
#pos=nx.get_node_attributes(G,'pos') # returns a dictionary of attributes keyed by node


if len(list_focus_univ)==1:
    filename=str(list_focus_univ).replace("[","").replace("[","").replace('"','')
    plot_title=str(list_focus_univ).replace("[","").replace("[","").replace('"','')
else:
    filename="multiple universities"
    plot_title="multiple universities"

dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

p=nx.single_source_shortest_path_length(G,ncenter)





edge_trace = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' | Viridis
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='Viridis',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='degree',#title='degree',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))






for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)

    
    
    
for node, adjacencies in enumerate(G.adjacency_list()):
    node_trace['marker']['color'].append(len(adjacencies))   # for the nodecolor to be degree
    #node_trace['marker']['color'].append(int(G.node[node]['edits']))  # for the node color to be number of edits
    node_info = 'k: '+str(len(adjacencies))+"<br>" + str(G.node[node]['email'])+"<br>edits:" + str(G.node[node]['edits']) +"<br>adds:" + str(G.node[node]['adds']) +"<br>dels:" + str(G.node[node]['deletes'])  #str(new_new_dict[node])
    
    
    ### or node_info = 'position: '+str(dict_pos[node])          
    node_trace['text'].append(node_info)




fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title=plot_title,
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    text="",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))



print "N:",len(G.nodes()),"  L:", len(G.edges())


iplot(fig)#, filename=filename)


N: 3773   L: 16476


In [38]:
#filename=str(list_focus_univ).replace("[","").replace("]","").replace(",","-").replace('"','')+GC_string
py.image.save_as(fig, filename='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/multiple_universities_color_k'+GC_string+'.png')


KeyError: 'error'

In [ ]:
with open(path+"first_network_"+focus_university+".pickle",'wb') as f:
    %time    pickle.dump(G, f)
print "written:",path+"first_network_"+focus_university+".pickle\n"




with open(path+"dict_position_first_network_"+focus_university+".pickle",'wb') as f:
    %time    pickle.dump(dict_pos, f)
print "written:",path+"dict_position_first_network_"+focus_university+".pickle"
